In [1]:
import numpy as np
import cv2

In [2]:
# Retrieved example ply_header from http://paulbourke.net/dataformats/ply/example1.ply

# The PLY format encodes an object as a collection of vertices, faces and other elements, and other 
# important properties like oclor and normal direction. Some of the objects include:
# polygon objects from modeling programs, range data, terrain data, and more. 
ply_header = '''ply
                format ascii 1.0
                element vertex %(vert_num)d
                property float x
                property float y
                property float z
                property uchar red
                property uchar green
                property uchar blue
                end_header
                '''

In [1]:
# Code for exporting the 3d Image to a 3rd party readable format. What the format does is explained above.
# From https://stackoverflow.com/questions/46959106/f-write-unsupported-format-character-error-after-changing-input-data
def write_ply(fn, verts, colors):
    verts = verts.reshape(-1, 3)
    colors = colors.reshape(-1, 3)
    verts = np.hstack([verts, colors])
    with open(fn, 'wb') as f:
        f.write((ply_header % dict(vert_num=len(verts))).encode('utf-8'))
        np.savetxt(f, verts, fmt='%f %f %f %d %d %d ')

In [5]:
print('loading images...')
imgL = cv2.pyrDown(cv2.imread('left_aloe.jpg'))  # downscale images for faster processing
imgR = cv2.pyrDown(cv2.imread('right_aloe.jpg'))

# disparity range is tuned for 'aloe' image pair
window_size = 3
min_disp = -1
num_disp = 16

# https://docs.opencv2.org/3.4/d2/d85/classcv2_1_1StereoSGBM.html
stereo = cv2.StereoSGBM_create(minDisparity = min_disp,
    numDisparities = num_disp,
    blockSize = 15,
    P1 = 8 * 3 * window_size ** 2,
    P2 = 32 * 3 * window_size ** 2,
    disp12MaxDiff = 1,
    uniquenessRatio = 10,
    speckleWindowSize = 5,
    speckleRange = 5,
)

print('computing disparity...')
disp = stereo.compute(imgL, imgR).astype(np.float32) / 16.0

Q = np.float32([[1, 0, 0, -0.5 * imgL.shape[:2][0]],
                [0,-1, 0, 0.5 * imgL.shape[:2][1]],
                [0, 0, 0, -0.8 * w],
                [0, 0, 1, 0]])

points = cv2.reprojectImageTo3D(disp, Q)
colors = cv2.cvtColor(imgL, cv2.COLOR_BGR2RGB)

mask = disp > disp.min()
out_points = points[mask]
out_colors = colors[mask]

out_fn = 'out.ply'
write_ply(out_fn, out_points, out_colors)
print('%s saved' % out_fn)

cv2.imshow('left', imgL)
cv2.imshow('disparity', (disp-min_disp)/num_disp)
cv2.waitKey()

print('Done')

loading images...
computing disparity...
generating 3d point cloud...
out.ply saved
Done
